In [1]:
from qepc.utils.paths import get_project_root
import sys

PROJECT_ROOT = get_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print('PROJECT_ROOT:', PROJECT_ROOT)


PROJECT_ROOT: C:\Users\wdors\qepc_project
qepc in root? True


In [2]:
from qepc.brain.games_loader import fetch_league_games, build_games_table
from qepc.brain.boxscores_loader import fetch_boxscores_for_games
from qepc.brain.state_vectors import build_rolling_player_state_vectors

season = "2023-24"

# 1) Games table
team_games = fetch_league_games(season)
games_df = build_games_table(team_games)

print("games_df rows:", len(games_df))
display(games_df.head())

# 2) Take a manageable subset of GAME_IDs to start
game_ids = games_df["GAME_ID"].unique().tolist()[:100]  # first 100 games

print("Number of GAME_IDs we'll use:", len(game_ids))

trad_df, _ = fetch_boxscores_for_games(game_ids, sleep_seconds=0.7, verbose=True)

print("trad_df shape:", trad_df.shape)
display(trad_df.head())


[games_loader] Fetching LeagueGameLog for season=2023-24, season_type=Regular Season...
[games_loader] Retrieved 2460 team-games.
games_df rows: 1230


,GAME_ID,GAME_DATE,SEASON_ID,HOME_TEAM_ID,HOME_TEAM_NAME,HOME_TEAM_ABBREVIATION,HOME_PTS,AWAY_TEAM_ID,AWAY_TEAM_NAME,AWAY_TEAM_ABBREVIATION,AWAY_PTS,HOME_RESULT,MARGIN,TOTAL_POINTS
0,0022300061,2023-10-24,22023,1610612743,Denver Nuggets,DEN,119,1610612747,Los Angeles Lakers,LAL,107,W,12,226
1,0022300062,2023-10-24,22023,1610612744,Golden State Warriors,GSW,104,1610612756,Phoenix Suns,PHX,108,L,-4,212
2,0022300070,2023-10-25,22023,1610612741,Chicago Bulls,CHI,104,1610612760,Oklahoma City Thunder,OKC,124,L,-20,228
3,0022300074,2023-10-25,22023,1610612746,LA Clippers,LAC,123,1610612757,Portland Trail Blazers,POR,111,W,12,234
4,0022300068,2023-10-25,22023,1610612748,Miami Heat,MIA,103,1610612765,Detroit Pistons,DET,102,W,1,205


Number of GAME_IDs we'll use: 100
[boxscores_loader] (1) Fetching boxscores for GAME_ID=0022300061...
[boxscores_loader] (2) Fetching boxscores for GAME_ID=0022300062...
[boxscores_loader] (3) Fetching boxscores for GAME_ID=0022300070...
[boxscores_loader] (4) Fetching boxscores for GAME_ID=0022300074...
[boxscores_loader] (5) Fetching boxscores for GAME_ID=0022300068...
[boxscores_loader] (6) Fetching boxscores for GAME_ID=0022300067...
[boxscores_loader] (7) Fetching boxscores for GAME_ID=0022300065...
[boxscores_loader] (8) Fetching boxscores for GAME_ID=0022300066...
[boxscores_loader] (9) Fetching boxscores for GAME_ID=0022300071...
[boxscores_loader] (10) Fetching boxscores for GAME_ID=0022300064...
[boxscores_loader] (11) Fetching boxscores for GAME_ID=0022300073...
[boxscores_loader] (12) Fetching boxscores for GAME_ID=0022300069...
[boxscores_loader] (13) Fetching boxscores for GAME_ID=0022300072...
[boxscores_loader] (14) Fetching boxscores for GAME_ID=0022300063...
[boxscore

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints,GAME_ID
0,0022300061,1610612743,Denver,Nuggets,DEN,nuggets,1629008,Michael,Porter Jr.,M. Porter Jr.,...,10,12,2,2,0,0,1,12,12.0,0022300061
1,0022300061,1610612743,Denver,Nuggets,DEN,nuggets,203932,Aaron,Gordon,A. Gordon,...,5,7,5,2,1,0,0,15,6.0,0022300061
2,0022300061,1610612743,Denver,Nuggets,DEN,nuggets,203999,Nikola,Jokić,N. Jokić,...,10,13,11,1,1,2,2,29,15.0,0022300061
3,0022300061,1610612743,Denver,Nuggets,DEN,nuggets,203484,Kentavious,Caldwell-Pope,K. Caldwell-Pope,...,1,2,1,3,1,3,5,20,10.0,0022300061
4,0022300061,1610612743,Denver,Nuggets,DEN,nuggets,1627750,Jamal,Murray,J. Murray,...,2,2,6,0,1,1,3,21,3.0,0022300061


In [3]:
from qepc.brain.state_vectors import build_rolling_player_state_vectors

rolling_state_df = build_rolling_player_state_vectors(
    boxscores_df=trad_df,
    games_df=games_df,
    window=10,
)

print("rolling_state_df shape:", rolling_state_df.shape)
display(rolling_state_df.head())


rolling_state_df shape: (2765, 36)


,PLAYER_ID,TEAM_ID,GAME_ID,GAME_DATE,roll_mean_minutes_10,roll_std_minutes_10,roll_mean_points_10,roll_std_points_10,roll_mean_fieldGoalsMade_10,roll_std_fieldGoalsMade_10,...,roll_mean_assists_10,roll_std_assists_10,roll_mean_steals_10,roll_std_steals_10,roll_mean_blocks_10,roll_std_blocks_10,roll_mean_turnovers_10,roll_std_turnovers_10,roll_mean_foulsPersonal_10,roll_std_foulsPersonal_10
0,2544,1610612747,0022300061,2023-10-24,29.016667,0.000000,21.0,0.000000,10.000000,0.000000,...,5.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
1,2544,1610612747,0022300076,2023-10-26,32.008333,4.230856,21.0,0.000000,8.500000,2.121320,...,7.000000,2.828427,1.5,0.707107,1.000000,1.414214,2.500000,3.535534,1.000000,0.000000
2,2544,1610612747,0022300100,2023-10-29,34.366667,5.063129,23.0,3.464102,9.333333,2.081666,...,7.333333,2.081666,1.0,1.000000,0.666667,1.154701,4.333333,4.041452,1.333333,0.577350
3,2544,1610612747,0022300111,2023-10-30,33.970833,4.209147,22.0,3.464102,8.750000,2.061553,...,6.500000,2.380476,1.5,1.290994,0.750000,0.957427,4.500000,3.316625,1.000000,0.816497
4,2544,1610612747,0022300127,2023-11-01,35.673333,5.270695,24.6,6.542171,9.600000,2.607681,...,6.600000,2.073644,1.4,1.140175,1.000000,1.000000,4.400000,2.880972,1.400000,1.140175


In [4]:
from qepc.brain.players_loader import fetch_league_player_season_stats

season = "2023-24"

# Get the same season-level player stats we used before
player_stats_df = fetch_league_player_season_stats(season, measure_type="Base")

# Keep just the label columns we care about
label_cols = ["PLAYER_ID", "PLAYER_NAME", "TEAM_ID", "TEAM_ABBREVIATION"]
label_cols = [c for c in label_cols if c in player_stats_df.columns]

player_labels = player_stats_df[label_cols].drop_duplicates(subset=["PLAYER_ID"])

# Join labels onto the rolling state table
rolling_pretty = rolling_state_df.merge(
    player_labels,
    on="PLAYER_ID",
    how="left",
)

# Reorder columns so labels show up first
front_cols = ["PLAYER_NAME", "TEAM_ABBREVIATION", "PLAYER_ID", "TEAM_ID", "GAME_ID", "GAME_DATE"]
front_cols = [c for c in front_cols if c in rolling_pretty.columns]

other_cols = [c for c in rolling_pretty.columns if c not in front_cols]

rolling_pretty = rolling_pretty[front_cols + other_cols]

display(rolling_pretty.head())


,PLAYER_NAME,TEAM_ABBREVIATION,PLAYER_ID,GAME_ID,GAME_DATE,TEAM_ID_x,roll_mean_minutes_10,roll_std_minutes_10,roll_mean_points_10,roll_std_points_10,...,roll_std_assists_10,roll_mean_steals_10,roll_std_steals_10,roll_mean_blocks_10,roll_std_blocks_10,roll_mean_turnovers_10,roll_std_turnovers_10,roll_mean_foulsPersonal_10,roll_std_foulsPersonal_10,TEAM_ID_y
0,LeBron James,LAL,2544,0022300061,2023-10-24,1610612747,29.016667,0.000000,21.0,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.610613e+09
1,LeBron James,LAL,2544,0022300076,2023-10-26,1610612747,32.008333,4.230856,21.0,0.000000,...,2.828427,1.5,0.707107,1.000000,1.414214,2.500000,3.535534,1.000000,0.000000,1.610613e+09
2,LeBron James,LAL,2544,0022300100,2023-10-29,1610612747,34.366667,5.063129,23.0,3.464102,...,2.081666,1.0,1.000000,0.666667,1.154701,4.333333,4.041452,1.333333,0.577350,1.610613e+09
3,LeBron James,LAL,2544,0022300111,2023-10-30,1610612747,33.970833,4.209147,22.0,3.464102,...,2.380476,1.5,1.290994,0.750000,0.957427,4.500000,3.316625,1.000000,0.816497,1.610613e+09
4,LeBron James,LAL,2544,0022300127,2023-11-01,1610612747,35.673333,5.270695,24.6,6.542171,...,2.073644,1.4,1.140175,1.000000,1.000000,4.400000,2.880972,1.400000,1.140175,1.610613e+09
